Generative AI aims to generate new data from existing data such that the newly generated data does not exist in the world.

# Making a chatbot

**Installing all the necessary libraries**

In [ ]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 82.0 MB/s eta 0:00:00


In [ ]:
!uv pip install python-dotenv
!uv pip install duckduckgo-search requests beautifulsoup4 gradio

Using Python 3.12.12 environment at: /usr
Audited 1 package in 120ms
Using Python 3.12.12 environment at: /usr
Audited 4 packages in 111ms


**Importing the libraries**

In [ ]:
import requests
from bs4 import BeautifulSoup
import gradio as gr
from huggingface_hub import InferenceClient
from duckduckgo_search import DDGS

In [ ]:
# creating a chatbot which scrapes a url and then answers the questions regarding that webpage
from google.colab import userdata
LLAMA_SECRET_TOKEN = userdata.get('LLAMA_CHATBOT')
client = InferenceClient(
    model="meta-llama/Meta-Llama-3-70B-Instruct",
    token=LLAMA_SECRET_TOKEN
)

chat_history = []

In [ ]:
def fetch_website_info(url):
  try:
    response = requests.get(url, timeout=5)
    soup = BeautifulSoup(response.content, "html.parser")
    text = soup.get_text(separator = ' ', strip = True)
    return text[:1000]
  except Exception as e:
    return f"Could not fetch the website: {e}"

In [ ]:
# function to perform live web search
def search_query(query):
  try:
    results = []
    with DDGS() as ddgs:
      for r in ddgs.text(query, max_results=2):
        results.append(f"{r['title']}: {r['href']}") # title
    return "\n".join(results) if results else "No search results found"
  except Exception as e:
    return f"Error in searching: {e}"

In [ ]:
# main chatbot function
def chat_with_ai(user_input):
  global chat_history

  if not user_input.strip():
    return "Please enter a valid input."

  # add system prompt once
  if not chat_history:
    chat_history.append({"role":"system","content":"You are a smart, friendly and helpful assistant with access to live search and website reading."})

  # Agent Decision: if query has URL
  if 'http' in user_input:
    for word in user_input.split():
      if word.startswith("http"):
        site_data = fetch_website_info(word)
        user_input += f"\n\n[Website Data Extracted: {site_data}]"
  elif any(keyword in user_input.lower() for keyword in ["latest", "google", "duckduckgo", "now", "today", "tomorrow"]):
    search_results = search_query(user_input)
    user_input += f"\n\n[Search Results: {search_results}]"

  chat_history.append({"role":"user","content":user_input})

  try:
    resp = client.chat_completion(
        messages=chat_history,
        max_tokens = 500,
        temperature=0.7
      )
    assistant_msg = resp.choices[0].message.content
  except Exception as e:
    return f"Error: {str(e)}"

  chat_history.append({"role":"assistant","content":assistant_msg})
  return assistant_msg

In [ ]:
# gradio interface
gr.Interface(
    fn=chat_with_ai,
    inputs = gr.Textbox(lines=2, placeholder="Ask anything... (e.g., 'latest price of iPhone 17) or paste any link"),
    outputs = gr.Textbox(label="AI Response", lines=10),
    title="Smart AI Chatbot (LLaMa3 + Live Search + Website Reader)",
    description="Chat with AI which can search the web and read websites for updates."
).launch(
    share=True
)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f2dbf52d4f21d0d8da.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
